# get image description from gpt4

In [1]:
import json
import os
from openai import OpenAI

client = OpenAI(api_key="")

default_prompt = """
Assuming you are a store assistant please analyze the item in the picture.
 Only return the data in a json with the keys shape, dominant_colors, style, description, material, suggested_title, theme. 
 Please restrict the product description to 100 words. 
 For theme please return any special characteristics like nature, geometric, abstract etc. 
"""


def get_openai_opinion(url: str, image_capture_prompt=default_prompt) -> dict:
    print(f'Sending request for {url}')
    response = client.chat.completions.create(
        model="gpt-4-vision-preview",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": image_capture_prompt},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": url,
                        },
                    },
                ],
            }
        ],
        max_tokens=400,
    )
    print('Received response')
    json_resp = response.choices[0].message.content
    tokens_used = response
    try:
        # print(f'Processing content {json_resp}')
        json_resp = json_resp.replace('```json', '').replace('```', '')
        data = json.loads(json_resp)
        # print(f'Response obtained: {data}')
        return data
    except Exception as e:
        print('Error calling openai', e)
        return dict()

In [2]:
repo = 'https://raw.githubusercontent.com/zhangx306/moondream/main/rug_jpeg/'

res = {}
failed = {}
for file in os.listdir('rug_jpeg'):
    url =  repo + file
    print('\n', file)
    try:
        data = get_openai_opinion(url)
        res[file] = data
    except Exception as e:
        print(f'{file} failed')
        failed[file] = e

res


 9701 ARIANA TRIBAL 2.9 X 11.9.jpeg
Sending request for https://raw.githubusercontent.com/zhangx306/moondream/main/rug_jpeg/9701 ARIANA TRIBAL 2.9 X 11.9.jpeg
9701 ARIANA TRIBAL 2.9 X 11.9.jpeg failed

 Astro Plum.jpeg
Sending request for https://raw.githubusercontent.com/zhangx306/moondream/main/rug_jpeg/Astro Plum.jpeg
Astro Plum.jpeg failed

 Dune.jpeg
Sending request for https://raw.githubusercontent.com/zhangx306/moondream/main/rug_jpeg/Dune.jpeg
Dune.jpeg failed

 Circles – Blue.jpeg
Sending request for https://raw.githubusercontent.com/zhangx306/moondream/main/rug_jpeg/Circles – Blue.jpeg
Circles – Blue.jpeg failed

 17627 VINTAGE  5.8 X 8.5..jpeg
Sending request for https://raw.githubusercontent.com/zhangx306/moondream/main/rug_jpeg/17627 VINTAGE  5.8 X 8.5..jpeg
17627 VINTAGE  5.8 X 8.5..jpeg failed

 8623 – Grass.jpeg
Sending request for https://raw.githubusercontent.com/zhangx306/moondream/main/rug_jpeg/8623 – Grass.jpeg
8623 – Grass.jpeg failed

 17630 VINTAGE  4.5 X 6.3.j

{}

In [3]:
failed

{'9701 ARIANA TRIBAL 2.9 X 11.9.jpeg': openai.AuthenticationError("Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-5TRgw***************************************dz3X. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}"),
 'Astro Plum.jpeg': openai.AuthenticationError("Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-5TRgw***************************************dz3X. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}"),
 'Dune.jpeg': openai.AuthenticationError("Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-5TRgw***************************************dz3X. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}"),
 'Circles